## 3.Class 类

一句话核心思想——**“Duck type”**

简单来说，Python在设计程序时并不关心你扔进变量的是什么类型，

只要他能干活就好。

当你有个处理鸭子的函数。他须要一个扁嘴巴的属性和一个嘎嘎叫的方法。

你可以往里扔绿头鸭，嘴巴是扁的，会嘎嘎叫；

你也可以往里扔唐老鸭，嘴巴是扁的，也会嘎嘎叫；

你往里扔个川建国，只要嘴巴是扁的，能嘎嘎叫，也能当鸭子用。

也就是说，重要的不是具体的类型，而是“接口”。


## 3.1 从基础开始

In [36]:
class Duck():
    def __init__(self):  ## 定义方法的时候第一个参数必须是self，
                         ## 类似于this指针的感觉。
        self.flatMouth = True
    
    def cry(self):
        print("quack! quack!")

class DonaldDuck():
    def __init__(self):
        self.flatMouth = True

    def cry(self):
        print("quack! quack! from Donald duck")

class DonaldTrump():
    def __init__(self):
        self.flatMouth = True
    
    def cry(self):
        print("No one quacks better than me!")

def makeDucksCry(duck):
    if duck.flatMouth:
        duck.cry()

duck1 = Duck()         # 新建一只鸭子
duck2 = DonaldDuck()   # 新建一只唐老鸭
duck3 =  DonaldTrump() # 向川建国致以崇高的敬意

makeDucksCry(duck1)
makeDucksCry(duck2)
makeDucksCry(duck3)

quack! quack!
quack! quack! from Donald duck
No one quacks better than me!


### Duck Type的优点
* 不用写共通的父类 ~~偷懒成功~~
* 省掉父类之后，用父类指针指向子类对象会调用哪个方法的复杂问题也不用考虑了。
* 关注如何解决问题，而不是怎么去设计一个类 ~~去TMD UML~~

## 3.2 Mixin 混入类
如果你须要一些动物，一些能把自己写进文件；另一些把自己通过socket发出去；

而这些动物可能是鸭子，也可能是狗，还可能是其他的动物，你会怎么办？

C++程序员会把动物写成父类，然后派生子类；把写文件和socket也写成父类或者纯虚类，

然后抄起多重继承展现炉火纯青的OO技术。

JAVA程序员没有那么幸运，因为没有多重继承，他们只能把写文件和socket做成接口，

在各个子类上写具体的实现。




In [37]:
# 写文件Mixin
class FileOutputMixin():
    def getContent(self):
        # 定义一个接口
        raise NotImplementedError('Please implement getFIleContent')

    def saveToFile(self):
        print("\n假装把以下内容写进文件:\n%s"%self.getContent())

# 网络通信Mixin
class SendBySocketMixin():
    # 根据dry原则，其实这边应该统一写个纯虚类的，凑合着看=-=。。。。
    def getContent(self):
        # 定义一个接口
        raise NotImplementedError('Please implement getFIleContent')

    def sendToAddr(self, addr):
        print("\n假装把以下内容通过socket发到（%s）:\n%s"%(addr, self.getContent()))

#强化版川建国
class SuperDonaldTrump(DonaldTrump, SendBySocketMixin, FileOutputMixin):
    def getContent(self):
        return "I AM Trump, I have super power!"

duck4 = SuperDonaldTrump()

makeDucksCry(duck4)
duck4.saveToFile()
duck4.sendToAddr('The Sun')


No one quacks better than me!

假装把以下内容写进文件:
I AM Trump, I have super power!

假装把以下内容通过socket发到（The Sun）:
I AM Trump, I have super power!


mixin可以看成为插件，可以用少量的代价，为原本的类提供之所前不具备的能力。

为进一步的偷懒打下坚实的基础。

**偷懒首要原则：**

Dry = “Don't Repeat Yourself”

不要复制黏贴几乎一模一样的代码。

不要复制黏贴几乎一模一样的代码。

不要复制黏贴几乎一模一样的代码。

使用Mixin之后，使用isinstance函数可以快速判断某只鸭子是不是你想要的鸭子。

In [38]:
def sendToSun(duck):
    if isinstance(duck, SendBySocketMixin):
        print('\n%s is ready for launch!'% duck.__class__)
        duck.sendToAddr('sun')
    else:
        print('\n%s can not be sent to sun.'% duck.__class__)

sendToSun(duck3) #普通川建国
sendToSun(duck4) #强化川建国


<class '__main__.DonaldTrump'> can not be sent to sun.

<class '__main__.SuperDonaldTrump'> is ready for launch!

假装把以下内容通过socket发到（sun）:
I AM Trump, I have super power!


## 3.3 动态组装对象 & 反射

### 如果我须要量产去太阳的英雄该怎么办？

看下面的例子。

In [39]:
from types import MethodType
ducks = [duck1, duck2, duck3]

# 首先我们得把筛类型的筛子去掉
def sendToSunAlt(duck):
    # 反射
    # 拿到我们须要的发射程序。
    sendToAddr = getattr(duck, 'sendToAddr', None)
    
    # 确定他可以被调用
    if callable(sendToAddr):
        print('\n%s is ready for launch!'% duck.__class__)
        sendToAddr('sun')
    
    else:
        print('\n%s can not be sent to sun.'% duck.__class__)

# 定义一个通用的发射方法。
def sendToAddrGeneric(self, addr):
    self.cry()
    print('I AM going to %s\n'%addr)

# 动态组装(骚操作)
heros = []
for i in ducks:
    i.sendToAddr = MethodType(sendToAddrGeneric, i)
    heros.append(i)

# 一起去太阳
for hero in heros:
    sendToSunAlt(hero)

sendToSunAlt(duck4) #强化版川建国同志自然也是没有问题的。


<class '__main__.Duck'> is ready for launch!
quack! quack!
I AM going to sun


<class '__main__.DonaldDuck'> is ready for launch!
quack! quack! from Donald duck
I AM going to sun


<class '__main__.DonaldTrump'> is ready for launch!
No one quacks better than me!
I AM going to sun


<class '__main__.SuperDonaldTrump'> is ready for launch!

假装把以下内容通过socket发到（sun）:
I AM Trump, I have super power!


标准OO，Duck Type，Mixin，动态组装，反射的结合使用为解决现实问题提供了几乎无限的灵活性。

也带来了难以阅读，难以理解，难以维护等等等等的问题。（偷懒的代价）

在偷懒的时候，请考虑一下以后接手的同事，或者是半年后的自己的感受，

毕竟太阳上并不是很好玩。
